In [118]:
!pip install pyvi
!git clone https://github.com/nam157/rnn_text_generation

fatal: destination path 'rnn_text_generation' already exists and is not an empty directory.


In [119]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout,BatchNormalization
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import pickle
import os
from pyvi import ViTokenizer
import string
from tqdm import tqdm
import re

In [120]:
filename = "/content/rnn_text_generation/hoaVangTrenCoXanh.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [121]:
def clean_document(raw_text):
    new_text = re.sub('[:;()?“”\*\-\"!]', ' ', raw_text)
    doc = ViTokenizer.tokenize(new_text)
    tokens = doc.split()
    tokens = [word for word in tokens if word]
    return tokens

In [122]:
# table = str.maketrans('', '', string.punctuation.replace("_", ""))

In [123]:
words = clean_document(raw_text)
print(words[:100])

['chú', 'đàn', 'bảo', 'tôi', 'con', 'xòe', 'tay', 'ra', 'cho', 'chú', 'xem', 'nào', 'tôi', 'co', 'những', 'ngón', 'tay', 'lại', ',', 'nắm', 'thật', 'chặt', 'và', 'giấu', 'ra', 'sau', 'lưng', 'tay', 'con', 'sạch', 'cơ_mà', '.', 'hồi', 'sáng', 'con', 'đã', 'rửa', 'tay', 'rồi', '.', 'chú', 'đàn', 'phì_cười', 'chú', 'có', 'định', 'khám', 'tay', 'con', 'đâu', '.', 'con', 'xòe', 'tay', 'ra', 'để', 'chú', 'xem', 'con', 'có', 'mấy', 'cái', 'hoa_tay', 'thôi', '.', 'đằng', 'sau', 'lưng', ',', 'hai', 'bàn_tay', 'tôi', 'lỏng', 'đi', '.', 'tôi', 'chìa', 'bàn_tay', 'trái', 'ra', 'trước', 'mặt', 'chú', 'đàn', ',', 'thắc_mắc', 'hoa_tay', 'là', 'gì', 'hở_chú', 'chú', 'đàn', 'dựng', 'mắt', 'nhìn', 'tôi', 'con', 'lớn', 'từng', 'này']


In [124]:
# đưa vào set lấy các trường hợp
words_set = sorted(list(set(words)))
#lấy index
char_to_int = dict((c, i) for i, c in enumerate(words_set))
int_to_char = dict((i, c) for i, c in enumerate(words_set))

In [125]:
n_chars = len(words)
n_vocab = len(words_set)
print( "Total Characters: ", n_chars)
print( "Total Vocab: ", n_vocab)

Total Characters:  15004
Total Vocab:  2119


In [126]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = words[i:i + seq_length]
	seq_out = words[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
n_patterns

14904

In [127]:
X = np.reshape(dataX, (len(dataX), seq_length, 1))
#co dan X
X = X / float((n_vocab))
#one hot encoding
y = np_utils.to_categorical(dataY)

In [128]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [129]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_8 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 256)               1024      
_________________________________________________________________
dense_4 (Dense)              (None, 2119)              544583    
Total params: 1,335,111
Trainable params: 1,334,599
Non-trainable params: 512
__________________________________________

In [130]:
filepath="weights-improvement-47-1.2219-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=150, batch_size=128, callbacks=callbacks_list)

Epoch 1/150
117/117 [==============================] - 7s 34ms/step - loss: 7.5504 - accuracy: 0.0130

Epoch 00001: loss improved from inf to 7.34709, saving model to weights-improvement-47-1.2219-bigger.hdf5
Epoch 2/150
117/117 [==============================] - 4s 34ms/step - loss: 6.3976 - accuracy: 0.0550

Epoch 00002: loss improved from 7.34709 to 6.28536, saving model to weights-improvement-47-1.2219-bigger.hdf5
Epoch 3/150
117/117 [==============================] - 4s 34ms/step - loss: 6.0855 - accuracy: 0.0564

Epoch 00003: loss improved from 6.28536 to 6.10048, saving model to weights-improvement-47-1.2219-bigger.hdf5
Epoch 4/150
117/117 [==============================] - 4s 34ms/step - loss: 5.9906 - accuracy: 0.0637

Epoch 00004: loss improved from 6.10048 to 6.00222, saving model to weights-improvement-47-1.2219-bigger.hdf5
Epoch 5/150
117/117 [==============================] - 4s 34ms/step - loss: 5.8603 - accuracy: 0.0668

Epoch 00005: loss improved from 6.00222 to 5.9070

In [136]:
import sys
filename = 'weights-improvement-47-1.2219-bigger.hdf5'
model.load_weights(filename)
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ' '.join([int_to_char[value] for value in pattern]), "\"")
for i in range(500):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    sys.stdout.write(result+' ')
    pattern.append(index)
    pattern = pattern[1:]
print ("\nDone.")

Seed:
" nhà con mận là một con chó già . mắt nó bị lòa , gần như_không nhìn thấy gì . tai và mũi nó không còn thính nhạy nữa , nhưng dĩ_nhiên tôi vào nhà thì nó vẫn biết . nó nằm yên dưới gầm giường , mông quay về phía tôi và vẫy đuôi chào tôi một_cách biếng_nhác . con vện giống_hệt chủ của nó . ba con mận cũng bị bênh thong_manh , tôi nghe ba tôi nói thế . trước đây , lúc mắt còn tinh , ông hành_nghề cắt tóc . ông là một trong hai "
người thợ cắt tóc trong làng . nhưng ông oách hơn , ông cắt tóc ở nhà , có treo tấm giấy các_tông trước cửa , ghi nguệch_ngoạc hớt tóc . ông thợ kia là người hớt_tóc dạo . một thời_gian dài , tôi cắt tóc chỗ ba con mận . nhưng rồi ba tôi không cho tôi cắt tóc ở chỗ ông nữa . một lần ông suýt cắt đứt tai người này , lần khác ông làm chảy_máu cằm người khác . đến một ngày , người làng thấy da ông ửng đỏ , rồi nửa tháng sau trên làng da đỏ hiện lên những vảy trắng . ông vừa cắt tóc vừa gãi . khi ông gãi , những vảy trắng tróc ra , bay khắp nơi như bụi phấn . 